# Variable类型与自动微分模块

Variable类型是PyTorch中的另一个变量类型，是Autograd（自动微分）模块对张量进一步封装实现的，支持自动求导。

## Variable对象与Tensor对象之间的转换

torch.autograd.Variable是PyTorch的核心类，它用于封装张量，并记录应用于张量上的操作。<br>
Variable类主要用于自动梯度计算，即自动求导。

在PyTorch 0.4.0及以后的版本中，torch.Tensor和torch.autograd.Variable已经合并，你可以直接在torch.Tensor上进行自动求导。<br>
以下是一个使用torch.Tensor进行自动求导的示例：

In [6]:
import torch

# 创建一个张量，并设置requires_grad=True来跟踪它的计算历史
x = torch.tensor([4.0], requires_grad=True)
print("x:", x)

# 定义一个函数
y = x * x
print("y:", y)

# 计算梯度
y.backward()

# 打印梯度
print(y.requires_grad)
print("x.grad:", x.grad)

x: tensor([4.], requires_grad=True)
y: tensor([16.], grad_fn=<MulBackward0>)
True
x.grad: tensor([8.])


## 控制梯度计算

### torch.no_grad()

In [4]:
with torch.no_grad():
    # 定义一个函数
    y = x * x
    print("y:", y)

print(y.requires_grad)

y: tensor([16.])
False


### no_grad()装饰器

In [9]:
z1 = x * 2
print(z1.requires_grad)

@torch.no_grad()
def doubler(x):
    return x * 2

z2 = doubler(x)
print(z2.requires_grad)

True
False


### enable_grad() 与 no_grad() 嵌套

**用函数 enable_grad() 配合 with 嵌套：**

In [13]:
x = torch.ones(2, 2, requires_grad=True)
with torch.no_grad():
    # 启用grad
    with torch.enable_grad():
        y = x * 2
print(y.requires_grad)

True


**enable_grad()装饰器：**

In [12]:
@torch.enable_grad()
def doubler(x):
    return x * 2

with torch.no_grad():
    # 被enable_grad装饰后，不再受no_grad限制
    z = doubler(x)
print(z.requires_grad)

True


**enable_grad()函数作用在没有requires_grad属性的Variable对象上时，将无效：**

In [14]:
x = torch.ones(2, 2)
with torch.enable_grad():
    y = x * 2
print(y.requires_grad)

False


### set_grad_enabled()

In [16]:
x = torch.ones(2, 2, requires_grad=True)

# 统一关闭梯度计算
torch.set_grad_enabled(False)
y = x * 2
print(y.requires_grad)

# 统一打开梯度计算
torch.set_grad_enabled(True)
y = x * 2
print(y.requires_grad)

False
True


## Variable对象的属性

### grad_fn

In [19]:
x = torch.ones(2, 2)
print(x.grad_fn)

None


In [20]:
m = x + 1
print(m.grad_fn)

None


In [30]:
x = torch.ones(2, 2, requires_grad=True)
m = x + 1
print(m.grad_fn)

In [36]:
x = torch.ones(2, 2, requires_grad=True)
y = x + 1

# 修改了grad_fn
z = y.mean()
z.backward()
print(x.grad, z)

tensor([[0.2500, 0.2500],
        [0.2500, 0.2500]]) tensor(2., grad_fn=<MeanBackward0>)


上面代码是关于`grad_fn`的解释：

1. x是一个叶子节点，它是由用户直接创建的，所以它的grad_fn是None。
2. m是通过加法操作创建的，所以它的grad_fn是一个AddBackward对象，这个对象表示加法操作的反向传播函数。
3. f是通过mean操作创建的，所以它的grad_fn是一个MeanBackward对象，这个对象表示mean操作的反向传播函数。
4. 当你调用f.backward()时，PyTorch会使用这些grad_fn来构建计算图，然后使用这个计算图来计算梯度。在这个过程中，grad_fn起到了关键的作用。
5. 最后，x.grad给出了f对x的梯度。这个梯度是通过计算图和链式法则计算得到的

### is_leaf

在PyTorch中，is_leaf是torch.Tensor的一个属性，它表示这个张量是否是计算图中的叶子节点。

在PyTorch的自动求导系统中，所有的计算操作都会形成一个计算图。<br>
在这个计算图中，叶子节点通常是用户创建的张量，非叶子节点是通过计算操作生成的张量。

In [24]:
x = torch.ones(2, 2, requires_grad=True)
print(x.is_leaf)
m = x + 2
print(m.is_leaf)

True
False


**在PyTorch的自动求导系统中，区分叶子节点和非叶子节点有几个重要的作用：**

- 内存管理：在反向传播过程中，为了节省内存，PyTorch会自动清除非叶子节点的梯度。这是因为非叶子节点的梯度在计算完成后通常不再需要，所以可以被立即清除。而叶子节点的梯度通常需要保留，因为它们通常对应于模型的参数，我们需要使用它们的梯度来更新参数。

- 梯度计算：只有叶子节点的梯度会被自动计算并存储。如果你试图访问非叶子节点的梯度，你会发现它是None。如果你需要保留非叶子节点的梯度，你可以使用retain_grad()函数。

- 计算跟踪：只有叶子节点会被计算跟踪。如果你创建一个张量并设置requires_grad=True，那么这个张量就是一个叶子节点，所有对它的操作都会被跟踪，用于后续的梯度计算。而非叶子节点则不会被跟踪。

总的来说，区分叶子节点和非叶子节点是PyTorch自动求导系统的一个重要部分，它对内存管理、梯度计算和计算跟踪都有重要作用。

### detach() 分离叶子节点

In [ ]:
x = torch.ones(2, 2, requires_grad=True)

# 运行后会报错
# 必须提前使用 detach() 分离叶子节点
x.numpy()

In [39]:
x.detach().numpy()

array([[1., 1.],
       [1., 1.]], dtype=float32)